In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import scipy

from sklearn.metrics import accuracy_score, mean_absolute_error
from functools import partial

from treeck import SplitTree
from treeck.xgb import addtree_from_xgb_model
from treeck.verifier import Verifier
from treeck.distributed import DistributedVerifier
from treeck.z3backend import Z3Backend as Backend
import z3

from dask.distributed import Client

In [ ]:
mat = scipy.io.loadmat("../data/mnist.mat")
X = mat["X"]
y = mat["y"].reshape((70000,))

num_examples = X.shape[0]
num_features = X.shape[1]

np.random.seed(111)
indices = np.random.permutation(num_examples)

m = int(num_examples*0.9)
Itrain = indices[0:m]
Itest = indices[m:]

In [ ]:
label = y==5
dtrain = xgb.DMatrix(X[Itrain], label[Itrain], missing=None)
dtest = xgb.DMatrix(X[Itest], label[Itest], missing=None)

params = {
    "objective": "binary:logistic",
    "tree_method": "exact",
    "max_depth": 6,
    "learning_rate": 0.4,
    "eval_metric": "error",
    "seed": 10,
}
#xgb.cv(params, dtrain,
#      num_boost_round = 10,
#      nfold=5, metrics={'error'}, seed=0,
#      callbacks=[xgb.callback.print_evaluation(show_stdv=True),
#                 xgb.callback.early_stop(3)])
model = xgb.train(params, dtrain, num_boost_round=40,
                  early_stopping_rounds=10,
                  evals=[(dtrain, "train"), (dtest, "test")])

In [ ]:
at = addtree_from_xgb_model(num_features, model)
at.base_score = 0.0
pred = model.predict(xgb.DMatrix(X), output_margin=True)
acc = accuracy_score(pred > 0.0, label)
print(f"accuracy: {acc}")
mae = mean_absolute_error(pred[:10000], at.predict(X[:10000]))
print(f"mae model difference {mae}")

In [ ]:
indices = np.random.permutation(num_examples)
instance_index = indices[np.argmax((y[indices] == 5))]
instance = X[instance_index, :]

fig, ax = plt.subplots()
ax.imshow(255-instance.reshape((28,28)), cmap='gray')
plt.show()

In [ ]:
def vfactory(instance, num_features, at, leaf):
    offset = 10
    max_sum_offset = 50
    v = Verifier(at, leaf, Backend())
    
    sum_constraint = 0
    for j, pixel in zip(range(num_features), instance):
        x = v.xvar(j)
        v.add_constraint((x > max(0, pixel-offset)) & (x < min(255, pixel+offset)))
        sum_constraint += z3.If(x.get()-pixel <= 0, pixel-x.get(), x.get()-pixel)
    v.add_constraint(sum_constraint < max_sum_offset)
    v.add_constraint(v.fvar() < 0.0)
    return v

vfac = partial(vfactory, instance, num_features)

with Client("tcp://localhost:30333") as client:
    client.restart()
    st = SplitTree(at, {})
    dv = DistributedVerifier(client, st, vfac,
            check_paths = True,
            saturate_workers_factor=2,
            stop_when_sat = False)

    dv.check()

In [ ]:
for domtree_node_id, res in dv.results.items():
    status = res.get("status", None)
    if "check_paths_time" in res:
        print("check_paths_time", res["check_paths_time"])
    if status is not None:
        print(f"{domtree_node_id}, {status}, {res.get('check_time', ''):.4f}")
        for fid, doms in dv._st.get_leaf_domains(domtree_node_id).items():
            print(f" {fid:3}: {str(doms):12}", end="")
        print()
    
    if status is not None and status.is_sat():
        fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(11,4))
        inst = np.array(res["model"]["xs"])
        d = sum(abs(inst-instance))
        im0 = ax0.imshow(255-inst.reshape((28,28)), cmap='gray')
        im1 = ax1.imshow(inst.reshape((28,28))-instance.reshape((28,28)))
        fig.colorbar(im0, ax=ax0)
        fig.colorbar(im1, ax=ax1)
        ax0.set_title(f"f={at.predict_single(instance):.4f}")
        ax1.set_title(f"f={res['model']['f']:.4f}=={at.predict_single(inst):.4f}, d={d:.2f}")
        plt.show()

In [ ]:
print(dv._st.domtree())

In [ ]:
from treeck.plot import TreePlot

g = TreePlot()
g.add_tree(dv._st.domtree())
g.g